<a href="https://colab.research.google.com/github/guruprasadsrinivasmurthy/Guru/blob/master/ULMFIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [253]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [254]:
from fastai.text import * 


In [255]:
path = untar_data(URLs.IMDB_SAMPLE)
path


PosixPath('/root/.fastai/data/imdb_sample')

In [256]:
#from google.colab import files

#uploaded = files.upload()

In [257]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('Tweets.csv')

df1['negativereason'] = df1['negativereason'].replace(np.nan, "")

df1 ['text'] =  df1 ['text'] +df1['negativereason']

df2 = pd.DataFrame(df1, columns= ['airline_sentiment', 'text'])

df2.to_csv (path/'text1.csv', index = False, header=True)


In [258]:
from fastai.text import * 

data_lm = TextLMDataBunch.from_csv(path,'text1.csv')
#
# Classifier model data
data_clas = TextClasDataBunch.from_csv(path,'text1.csv', vocab=data_lm.train_ds.vocab, bs=32)


In [259]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')


In [260]:
data_lm = load_data(path, 'data_lm_export.pkl')
data_clas = load_data(path, 'data_clas_export.pkl', bs=16)


In [261]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)


epoch,train_loss,valid_loss,accuracy,time
0,4.582171,3.866195,0.292498,00:26


In [262]:
learn.save_encoder('ft_enc')


In [263]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')


RNNLearner(data=TextClasDataBunch;

Train: LabelList (11711 items)
x: TextList
xxbos @united is a joke . xxmaj lots of angry people at xxup iah this xxunk xxmaj tell,xxbos @usairways what is going on with the computers ? xxmaj why is my flight grounded ? xxmaj why does your airline suck so much ? # xxunk xxmaj flight,xxbos @americanair i do n't want to waste any more of my time on your airline ( you 've already robbed 21 hours of my day ) but i definitely will . xxmaj can't xxmaj tell,xxbos @virginamerica what happened to direct flights from xxup sfo - xxup fll ? xxmaj looking in xxmaj may , only see connecting # help # xxunk xxmaj booking xxmaj problems,xxbos @united i sent the message . xxmaj let me know xxup asap .
y: CategoryList
negative,negative,negative,negative,neutral
Path: /root/.fastai/data/imdb_sample;

Valid: LabelList (2929 items)
x: TextList
xxbos @usairways # xxup phl why does it take an hour to offload bags , especially after taking 30 + min to get a jetway to get off 

In [264]:
data_clas.show_batch()


text,target
xxbos @united xxmaj hi have a question re future xxmaj flight xxmaj booking xxmaj problems . xxup dub - xxup jac 29 / 9 xxup jac - xxup lax 8 / 10 xxup lax - xxup dub 13 / 10 . i 'm * xxup g. xxmaj what is checked bag allowance for xxup jac - xxup lax ?,neutral
"xxbos @united xxmaj united xxmaj club team is xxup a+ & & got me a seat xxmaj late xxmaj flightr . xxmaj still , not sure why a last min xxup ual xxmaj cancelled xxmaj flightlation costs me $ yet overbooked folks get $ ? xxmaj flight xxmaj booking xxmaj problems",negative
xxbos @southwestair xxmaj is your xxup bwi - xxup sjd service xxunk ? xxmaj was n't part of extension . xxmaj called intl desk . xxmaj they did n't know . xxmaj want to fly in xxmaj sept on xxmaj sat . xxmaj customer xxmaj service xxmaj issue,negative
xxbos @americanair why xxmaj cancelled xxmaj flight # xxunk on 6 / 25 & & 6 / 28 ? xxmaj flights now 4 hrs longer & & have layovers . xxmaj too xxmaj late xxmaj flight to find a xxunk priced alternative . xxmaj cancelled xxmaj flight,negative
"xxbos @americanair xxmaj as i already have a booked flight that i ca n't use & & ca n't change w / o a $ 200 fee , xxmaj flight xxmaj booking xxmaj problems a different type of ticket ... 1 / xxunk xxmaj booking xxmaj problems",negative


In [265]:
learn.fit_one_cycle(1, 1e-2)


epoch,train_loss,valid_loss,accuracy,time
0,0.370742,0.218110,0.910550,00:43


In [266]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))


epoch,train_loss,valid_loss,accuracy,time
0,0.343009,0.200601,0.922158,00:46


In [274]:
learn.predict("@VirginAmerica Can you find us a flt out of LAX that is sooner than midnight on Monday? That would be great customer service 😃")


(Category tensor(1), tensor(1), tensor([0.1986, 0.5173, 0.2841]))